In [1]:
import pandas as pd
import rdflib

g=rdflib.Graph()

g.load('recipe.owl', format='ttl')
g.load('neapolitan_pizza.ttl', format='ttl')

sparql_prefixes = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX neapolitan_pizza: <http://www.example.org/neapolitan_pizza#>
BASE <http://www.semanticweb.org/tpfliss/ontologies/2020/0/recipes#>
"""

In [2]:
recipes = pd.DataFrame(g.query(sparql_prefixes + """
SELECT DISTINCT ?recipe
WHERE {
    [] a :Recipe ;
    rdfs:label ?recipe .
}
""")).fillna('')

recipes

,0
0,neapolitan pizza dough


In [3]:
ingredients = pd.DataFrame(g.query(sparql_prefixes + """
SELECT DISTINCT ?amount ?unit ?ingredient ?comment
WHERE {
    [] a :Recipe ;
       :ingredient_list_item ?m .
    ?m :ingredient [ rdfs:label ?ingredient ] .
    OPTIONAL {
        ?m :amount ?amount .
    } OPTIONAL {
        ?m :unit [ rdfs:label ?unit ] .
    } OPTIONAL {
        ?m rdfs:comment ?comment .
    }
}
""")).fillna('')

ingredients

,0,1,2,3
0,4.5,cup,flour,
1,1,teaspoon,yeast,
2,1.75,teaspoon,salt,
3,0.25,cup,olive oil,
4,1.75,cup,water,
5,,,cornmeal,for dusting


Note in the future we'll want to handle modifiers (large metal spoon)
Also currently recipe step ingredients are pointing to the ingredient class rather than to specific measurement instance from the ingredients list (or even a source portion divided out from the original ingredient list).

We'll want SHACL restrictions to verify the recipe model is valid, and also probably want a tool to generate a recipe model from a DSL or csv lists.

Note the use of property paths on ingredient to override rdfs label (maybe not a great idea?)

In [4]:
steps = pd.DataFrame(g.query(sparql_prefixes + """
SELECT DISTINCT ?order ?sub_order ?operation ?ingredient ?product ?equipment ?description ?comment
WHERE {
    {
        ?s a :Step ;
            rdfs:label ?description ; # suppress steps w/ substeps
            :order ?order .
    } UNION {
        ?sg a :Step ;
            :order ?order ;
            :sub_step ?s .
        ?s :order ?sub_order .
    }
    OPTIONAL {
        ?s rdfs:label ?description .
    }
    OPTIONAL {
        ?s :operation [ rdfs:label ?operation ] .
    }
    OPTIONAL {
        ?s :ingredient+ [ rdfs:label ?ingredient ] .
    }
    OPTIONAL {
        ?s :product [ rdfs:label ?product ] .
    }
    OPTIONAL {
        ?s ?eq [ a [ rdfs:label ?equipment ] ] .
        ?eq rdfs:subPropertyOf* :equipment .
    }
    OPTIONAL {
        ?s rdfs:comment ?comment .
    }
}
ORDER BY ?order ?sub_order
"""), columns=['order', 'sub_order', 'operation', 'ingredient', 'product', 'equipment', 'description', 'comment']).fillna('')
steps.set_index(['order', 'sub_order'], inplace=True)

def collapse_column_as_string_list(df, grp, col):
    df[col] = df.groupby(grp)[col].apply(', '.join)

    return  steps.drop_duplicates()

# Might treat these as sub-steps in the future?
steps = collapse_column_as_string_list(steps, ['order', 'sub_order'], 'ingredient')
steps = collapse_column_as_string_list(steps, ['order', 'sub_order'], 'product')
steps = collapse_column_as_string_list(steps, ['order', 'sub_order'], 'equipment')

# Getting duplicat values, hacky fix
for col in ['ingredient', 'product', 'equipment']:
    steps[col] = steps[col].apply(lambda x: list(set(x.split(', '))) if x else [])
 
steps

operation                   ingredient                product  \
order sub_order                                                                 
1                    stir  [flour, yeast, water, salt]                [dough]   
2     1              line                           []                     []   
      2              mist                  [olive oil]                     []   
      3          sprinkle                   [cornmeal]                     []   
      4              move                [mixed dough]                     []   
      5               cut                      [dough]  [six pieces of dough]   
      6             round        [six pieces of dough]   [six balls of dough]   
      7              move         [six balls of dough]                     []   
3     1              move         [six balls of dough]                     []   
      2              rest         [six balls of dough]                     []   
4     1                           [six balls of dough]                     []   
5     1              move                           []                     []   
      2           preheat                           []                     []   
6                                                   []                     []   
7                                                   []                     []   
8                                                   []                     []   
9                                                   []                     []   

                                    equipment  \
order sub_order                                 
1                               [bowl, spoon]   
2     1          [parchment paper, sheet pan]   
      2          [parchment paper, sheet pan]   
      3                             [counter]   
      4                             [counter]   
      5                             [scraper]   
      6                               [hands]   
      7                           [sheet pan]   
3     1             [refrigerator, sheet pan]   
      2             [refrigerator, sheet pan]   
4     1             [refrigerator, sheet pan]   
5     1                   [oven, pizza stone]   
      2                                [oven]   
6                                          []   
7                                          []   
8                                          []   
9                                          []   

                                                       description  \
order sub_order                                                      
1                 stir together the flour, salt, and instant yeast   
2     1                        Line sheet pan with parchment paper   
      2                                   mist the parchment paper   
      3          sprinkle flour on the counter and transfer the...   
      4                          transfer the dough to the counter   
      5                         cut the dough into six equal parts   
      6                                                              
      7                          transfer dough balls to sheet pan   
3     1                          put the pan into the refrigerator   
      2                                   rest the dough overnight   
4     1                     remove the dough from the refrigerator   
5     1                           place a baking stone in the oven   
      2                                           preheat the oven   
6                                                   toss the dough   
7                                                    top the pizza   
8                                                   bake the pizza   
9                                       remove pizza from the oven   

                                      comment  
order sub_order                                
1                                              
2     1                                        
      2              

Note need to model "six pieces of dough" as a product of step 2.4

Need to model "move" destination better in step 2.6

This is a rough draft of encoding and querying relative time information in the recipe.

In [5]:
timing = pd.DataFrame(g.query(sparql_prefixes + """
SELECT DISTINCT ?event1 ?amount ?unit ?direction ?event2
WHERE {
    ?e1 :at [
        ?offset ?e2 ;
        :amount ?amount ;
        :unit [ rdfs:label ?unit ] ;
    ] .
    ?offset rdfs:subPropertyOf :time_offset ;
        rdfs:label ?direction .
    ?e1 rdfs:label ?event1 .
    ?e2 rdfs:label ?event2 .
}
"""), columns=['event1', 'amount', 'unit', 'direction', 'event2']).fillna('')

timing

,event1,amount,unit,direction,event2
0,remove the dough from the refrigerator,2,hour,before,top the pizza
1,preheat the oven,45,minute,before,top the pizza


The following queries subproperties of equipment that can be used as more specific annotation while still allowing more general queries on what equipment is used in various steps of the recipe. This kind of query can also be useful for populating dropdowns.

In [6]:
timing = pd.DataFrame(g.query(sparql_prefixes + """
SELECT DISTINCT ?p ?l
WHERE {
    ?s ?p ?o .
    ?p rdfs:subPropertyOf* :equipment .
    ?p rdfs:label ?l
}
"""), columns=['p', 'l']).fillna('')

timing

,p,l
0,http://www.semanticweb.org/tpfliss/ontologies/...,in
1,http://www.semanticweb.org/tpfliss/ontologies/...,from
2,http://www.semanticweb.org/tpfliss/ontologies/...,onto
3,http://www.semanticweb.org/tpfliss/ontologies/...,into
